In [1]:
import nltk
from string import punctuation
from nltk.tokenize import RegexpTokenizer, word_tokenize

from unicodedata import normalize
from nltk.stem import RSLPStemmer
from nltk.util import ngrams
import pandas as pd

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('rslp')
import time
import pyterrier as pt
pt.init()


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Flavio\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Flavio\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\Flavio\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!
PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7



In [2]:
retriever = pt.BatchRetrieve("E:\Corpora_em_json\pyterrier\sem_pre_processamento",{"vmodel":"BM25","k1":1.5,"b":0.75})

JavaException: JVM exception occurred: No Manager implementation found for index E:\Corpora_em_json\pyterrier\sem_pre_processamento (IndexRef) - Do you need to import another package (terrer-core or terrier-rest-client)? Or perhaps the index location is wrong. Found builders were org.terrier.querying.LocalManager$Builder,org.terrier.restclient.RestClientManagerBuilder,org.terrier.querying.ThreadSafeManager$Builder java.lang.IllegalArgumentException

In [ ]:
query = "emenda"

In [ ]:
r = retriever.search(query)

In [ ]:
r

In [3]:
r.columns

NameError: name 'r' is not defined

In [2]:
import time

In [3]:
df_assunto= pd.read_table("dados-conle-anonimizado-assunto-notnull - dados-conle-anonimizado-assunto-notnull.tsv",sep=";")

In [4]:
arr_assunto = df_assunto.to_numpy()
y,X = arr_assunto[:,0],arr_assunto[:,1]
y = [i.strip() for i in y]

In [5]:
base = pd.read_csv("base_20230428_douglas.csv", delimiter=',', encoding="utf-8")

In [5]:
base.iloc[0,1]

'PRC 130/1997'

In [6]:
def verificar(y,top_n,top_k):
    for i in range(top_k):
        if len(top_n)>0:
            index = int(top_n.iloc[i,2])
            nome = base.iloc[index,4]
            if y in nome:
                return 1
        else:
            return 0
    return 0

In [7]:
def avaliacaoRecall(isPreprocess,top_k=20):
    
    quant_encontrado = 0
    quant_relevante =0
    antes = time.time()
    for l,x in zip(y,X):

        tokenized_query3 = x
        if isPreprocess:
            tokenized_query3 = preprocess(x)
        
        top_n = retriever.search(tokenized_query3)
    
        quant_relevante+=1
        quant_encontrado+=verificar(l,top_n,top_k)
    
    recall = quant_encontrado / quant_relevante
    depois = time.time()
    print("Recall: "+str(recall))
    print("Duração: %f" %(depois - antes))

In [8]:
def avaliacaoRR(isPreprocess):
    quant_relevante =0
    for l,x in zip(y,X):

        tokenized_query3 = x
        if isPreprocess:
            tokenized_query3 = preprocess(x)
        
        top_n = retriever.search(tokenized_query3)
        if len(top_n)>0:
            index = int(top_n.iloc[0,2])
            nome = base.iloc[index,4]
            if l.strip() in nome.strip():
                quant_relevante+=1
    rr = quant_relevante/295
    print("RR: %f" %(rr))

In [9]:
def avaliacaoPrecision(isPreprocess,top_k=20):
    
    quant_encontrado = 0
    soma =0
    for l,x in zip(y,X):

        tokenized_query3 = x
        if isPreprocess:
            tokenized_query3 = preprocess(x)
        
        top_n = retriever.search(tokenized_query3)
    
        
        quant_encontrado=verificar(l,top_n,top_k)
        soma+= (quant_encontrado / top_k)
    
    precision = soma / 295
    
    print("Precision: "+str(precision))

In [10]:
def avaliacaoMAP(isPreprocess,top_k=20):
    
    l_v = list()
    labels_nsL = list()
    for l,x in zip(y,X):

        tokenized_query3 = x
        if isPreprocess:
            tokenized_query3 = preprocess(x)
        
        top_n = retriever.search(tokenized_query3)
    
        l_v.append(l)
        
        if len(top_n)>0:
            labels_nsL.append([base.iloc[int(top_n.iloc[i,4]),1] for i in range(top_k)])
        else:
            labels_nsL.append(list())
            
    denominador = 0
    encontrou=False
    soma=0
    for Y,x in zip(l_v,labels_nsL):
        calc = 0
        quant=0
        if len(x)>0:
            for k in x:
                if str(k).strip() == Y.strip():
                    quant=1
                    encontrou=True
                denominador+=1
                calc=quant/denominador
                if encontrou:
                    calc = calc * 1
                else:
                    calc = calc * 0
                soma+=calc
                encontrou = False
    MAP = soma /295
    print("MAP@ %d: %f" % (top_k,MAP))
            

In [8]:
class Savoy:

    def __removeAllPTAccent(self, old_word):
        word = list(old_word)
        len_word = len(word)-1
        for i in range(len_word, -1, -1):
            if word[i] == 'ä':
                word[i] = 'a'
            if word[i] == 'â':
                word[i] = 'a'
            if word[i] == 'à':
                word[i] = 'a'
            if word[i] == 'á':
                word[i] = 'a'
            if word[i] == 'ã':
                word[i] = 'a'
            if word[i] == 'ê':
                word[i] = 'e'
            if word[i] == 'é':
                word[i] = 'e'
            if word[i] == 'è':
                word[i] = 'e'
            if word[i] == 'ë':
                word[i] = 'e'
            if word[i] == 'ï':
                word[i] = 'i'
            if word[i] == 'î':
                word[i] = 'i'
            if word[i] == 'ì':
                word[i] = 'i'
            if word[i] == 'í':
                word[i] = 'i'
            if word[i] == 'ü':
                word[i] = 'u'
            if word[i] == 'ú':
                word[i] = 'u'
            if word[i] == 'ù':
                word[i] = 'u'
            if word[i] == 'û':
                word[i] = 'u'
            if word[i] == 'ô':
                word[i] = 'o'
            if word[i] == 'ö':
                word[i] = 'o'
            if word[i] == 'ó':
                word[i] = 'o'
            if word[i] == 'ò':
                word[i] = 'o'
            if word[i] == 'õ':
                word[i] = 'o'
            if word[i] == 'ç':
                word[i] = 'c'

        new_word = "".join(word)
        return new_word

    def __finalVowelPortuguese(self, word):
        len_word = len(word)
        if len_word > 3:
            if word[-1] == 'e' or word[-1] == 'a' or word[-1] == 'o':
                word = word[:-1]

        return word

    def __remove_PTsuffix(self, word):
        len_word = len(word)

        if len_word > 3:
            if word[-1] == 's' and word[-2] == 'e' and (word[-3] == 'r' or word[-3] == 's' or word[-3] == 'z' or word[-3] == 'l'):
                word = word[:-2]
                return word
        if len_word > 2:
            if word[-1] == 's' and word[-2] == 'n':
                new_word = list(word)
                new_word[-2] = 'm'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 3:
            if (word[-1] == 's' and word[-2] == 'i') and (word[-3] == 'e' or word[-3] == 'é'):
                new_word = list(word)
                new_word[-3] = 'e'
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 3:
            if word[-1] == 's' and word[-2] == 'i' and word[-3] == 'a':
                new_word = list(word)
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 3:
            if word[-1] == 's' and word[-2] == 'i' and word[-3] == 'ó':
                new_word = list(word)
                new_word[-3] = 'o'
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 3:
            if word[-1] == 's' and word[-2] == 'i':
                new_word = list(word)
                new_word[-1] = 'l'
                sing = "".join(new_word)
                return sing

        if len_word > 2:
            if word[-1] == 's' and word[-2] == 'e' and word[-3] == 'õ':
                new_word = list(word)
                new_word[-3] = 'ã'
                new_word[-2] = 'o'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing
            if word[-1] == 's' and word[-2] == 'e' and word[-3] == 'ã':
                new_word = list(word)
                new_word[-2] = 'o'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 5:
            if word[-1] == 'e' and word[-2] == 't' and word[-3] == 'n' and word[-4] == 'e' and word[-5] == 'm':
                word = word[:-5]
                return word

        if len_word > 2:
            if word[-1] == 's':
                word = word[:-1]

        return word

    def __normFemininPortuguese(self, word):

        len_word = len(word)

        if len_word < 3 or word[-1] != 'a':
            return word

        if len_word > 6:

            if word[-2] == 'h' and word[-3] == 'n' and word[-4] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'c' and word[-3] == 'a' and word[-4] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'r' and word[-3] == 'i' and word[-4] == 'e':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

        if len_word > 5:
            if word[-2] == 'n' and word[-3] == 'o':
                new_word = list(word)
                new_word[-3] = 'ã'
                new_word[-2] = 'o'
                masc = "".join(new_word)
                masc = masc[:-1]
                return masc

            if word[-2] == 'r' and word[-3] == 'o':
                word = word[:-1]
                return word

            if word[-2] == 's' and word[-3] == 'o':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 's' and word[-3] == 'e':
                new_word = list(word)
                new_word[-3] = 'ê'
                masc = "".join(new_word)
                masc = masc[:-1]
                return masc

            if word[-2] == 'c' and word[-3] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'd' and word[-3] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'd' and word[-3] == 'a':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'v' and word[-3] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'm' and word[-3] == 'a':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'n':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

        return word

    def stem(self, word):
        len_word = len(word)
        if len_word > 2:
            word = self.__remove_PTsuffix(word)
            word = self.__normFemininPortuguese(word)
            word = self.__finalVowelPortuguese(word)
            word = self.__removeAllPTAccent(word)

        return word

In [9]:
class RSLP_S:
    def __plural_reduction(self, word):
        excep = ["lápis","cais","mais","crúcis","biquínis","pois","depois","dois","leis" ]
        excep_s = ["aliás","pires","lápis","cais","mais","mas","menos", "férias","fezes","pêsames","crúcis","gás", "atrás","moisés","através","convés","ês", "país","após","ambas","ambos","messias"]

        len_word = len(word)
        new_word = list(word)

        if len_word >= 3:
            if new_word[-1] == 's' and new_word[-2] == 'n':
                new_word[-2] = 'm'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

            if new_word[-1] == 's' and new_word[-2] == 'e' and new_word[-3] == 'õ':
                new_word[-3] = 'ã'
                new_word[-2] = 'o'
                sing = "".join(new_word)
                sing = sing[:-1]
                return  sing

            if new_word[-1] == 's' and new_word[-2] == 'e' and new_word[-3] == 'ã':
                if word == 'mães':
                    word = word[:-1]
                    return word
                else:
                    new_word[-2] = 'o'
                    sing = "".join(new_word)
                    sing = sing[:-1]
                    return sing

            if new_word[-1] == 's' and new_word[-2] == 'i' and new_word[-3] == 'a':
                if word != 'cais' and word != 'mais':
                    new_word[-2] = 'l'
                    sing = "".join(new_word)
                    sing = sing[:-1]
                    return sing

            if new_word[-1] == 's' and new_word[-2] == 'i' and new_word[-3] == 'é':
                new_word[-3] = 'e'
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

            if new_word[-1] == 's' and new_word[-2] == 'i' and new_word[-3] == 'e':
                new_word[-3] = 'e'
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

            if new_word[-1] == 's' and new_word[-2] == 'i' and new_word[-3] == 'ó':
                new_word[-3] = 'o'
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

            if new_word[-1] == 's' and new_word[-2] == 'i':
                if word not in excep:
                    new_word[-1] = 'l'
                    sing = "".join(new_word)
                    return sing

            if new_word[-1] == 's' and new_word[-2] == 'e' and new_word[-3] == 'l':
                word = word[:-2]
                return word

            if new_word[-1] == 's' and new_word[-2] == 'e' and new_word[-3] == 'r':
                word = word[:-2]
                return word

            if new_word[-1] == 's':
                if word not in excep_s:
                    word = word[:-1]

        return word

    def stem(self, word):
        word = self.__plural_reduction(word)

        return word

## Config 1

In [14]:
index_path = 'E:\Corpora_em_json\pyterrier\sem_pre_processamento'

In [15]:
retriever = pt.BatchRetrieve(index_path,{"vmodel":"BM25","k1":1.5,"b":0.75})

In [16]:
#def preprocess(txt):
#    tokenizer = pt.autoclass("org.terrier.indexing.tokenisation.Tokeniser").getTokeniser()
#    terms = tokenizer.getTokens(txt)
#    terms = " ".join(terms)
#    return terms
def preprocess(txt):
    txt = str(txt)
    tokenizer = pt.autoclass("org.terrier.indexing.tokenisation.Tokeniser").getTokeniser()
    terms = tokenizer.getTokens(txt)
    terms = " ".join(terms)
    return terms

## Avaliação

### Recall

In [16]:
avaliacaoRecall(True,5)

Recall: 0.4440677966101695
Duração: 41.252053


In [17]:
avaliacaoRecall(True,10)

Recall: 0.5186440677966102
Duração: 34.949999


In [19]:
avaliacaoRecall(True,20)

Recall: 0.5559322033898305
Duração: 37.252147


### Precision

In [21]:
avaliacaoPrecision(True,5)

Precision: 0.08135593220338964


In [22]:
avaliacaoPrecision(True,10)

Precision: 0.044745762711864305


In [23]:
avaliacaoPrecision(True,20)

Precision: 0.024576271186440617


### MAP

In [24]:
avaliacaoMAP(True,5)

MAP@ 5: 0.002801


In [25]:
avaliacaoMAP(True,10)

MAP@ 10: 0.001751


In [26]:
avaliacaoMAP(True,20)

MAP@ 20: 0.000937


### RR

In [27]:
avaliacaoRR(True)

RR: 0.277966


## Config 5

In [19]:
index_path = 'E:\Corpora_em_json\pyterrier\pre_processamento_lower_remocao_punctuation_acentuacao_stopwords'

In [20]:
retriever = pt.BatchRetrieve(index_path,{"vmodel":"BM25","k1":1.5,"b":0.75})

In [21]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [word for word in terms if word not in stopwords]
    terms = " ".join(terms)
    return terms

## Avaliação

### Recall

In [22]:
avaliacaoRecall(True,5)

Recall: 0.5932203389830508
Duração: 20.590930


In [23]:
avaliacaoRecall(True,10)

Recall: 0.6576271186440678
Duração: 16.027995


In [24]:
avaliacaoRecall(True,20)

Recall: 0.6915254237288135
Duração: 15.504034


### Precision

In [34]:
avaliacaoPrecision(True,5)

Precision: 0.10644067796610142


In [35]:
avaliacaoPrecision(True,10)

Precision: 0.05898305084745755


In [36]:
avaliacaoPrecision(True,20)

Precision: 0.03254237288135594


### MAP

In [37]:
avaliacaoMAP(True,5)

MAP@ 5: 0.003201


In [38]:
avaliacaoMAP(True,10)

MAP@ 10: 0.002464


In [39]:
avaliacaoMAP(True,20)

MAP@ 20: 0.001589


### RR

In [40]:
avaliacaoRR(True)

RR: 0.345763


## Config 8

In [10]:
index_path = 'E:\Corpora_em_json\pyterrier\pre_processamento_rslp_full'

In [11]:
retriever = pt.BatchRetrieve(index_path,{"vmodel":"BM25","k1":1.5,"b":0.75})

In [12]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLPStemmer()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
    return " ".join(terms)

## Avaliação

### Recall

In [ ]:
avaliacaoRecall(True,5)

In [ ]:
avaliacaoRecall(True,10)

In [13]:
avaliacaoRecall(True,20)

Recall: 0.7288135593220338
Duração: 21.223784


### Precision

In [47]:
avaliacaoPrecision(True,5)

Precision: 0.10576271186440651


In [48]:
avaliacaoPrecision(True,10)

Precision: 0.061355932203389786


In [49]:
avaliacaoPrecision(True,20)

Precision: 0.033220338983050865


### MAP

In [50]:
avaliacaoMAP(True,5)

MAP@ 5: 0.002977


In [51]:
avaliacaoMAP(True,10)

MAP@ 10: 0.002210


In [52]:
avaliacaoMAP(True,20)

MAP@ 20: 0.001553


### RR

In [53]:
avaliacaoRR(True)

RR: 0.366102


## Config 9

In [27]:
index_path = 'E:\Corpora_em_json\pyterrier\pre_processamento_savoy_full'

In [28]:
retriever = pt.BatchRetrieve(index_path,{"vmodel":"BM25","k1":1.5,"b":0.75})

In [29]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = Savoy()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
    return " ".join(terms)

## Avaliação

### Recall

In [ ]:
avaliacaoRecall(True,5)

In [ ]:
avaliacaoRecall(True,10)

In [30]:
avaliacaoRecall(True,20)

Recall: 0.7220338983050848
Duração: 23.918069


### Precision

In [60]:
avaliacaoPrecision(True,5)

Precision: 0.10711864406779634


In [61]:
avaliacaoPrecision(True,10)

Precision: 0.0603389830508474


In [62]:
avaliacaoPrecision(True,20)

Precision: 0.032711864406779666


### MAP

In [63]:
avaliacaoMAP(True,5)

MAP@ 5: 0.003111


In [64]:
avaliacaoMAP(True,10)

MAP@ 10: 0.001978


In [65]:
avaliacaoMAP(True,20)

MAP@ 20: 0.001317


### RR

In [66]:
avaliacaoRR(True)

RR: 0.355932


## Config 18

In [31]:
index_path = 'E:\Corpora_em_json\pyterrier\pre_processamento_unigram_bigram_rslp_full'

In [32]:
retriever = pt.BatchRetrieve(index_path,{"vmodel":"BM25","k1":1.5,"b":0.75})

In [33]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLPStemmer()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + "_" + w[1]
        ngram.append(string)
    
    return " ".join(ngram)

## Avaliação

### Recall

In [ ]:
avaliacaoRecall(True,5)

In [ ]:
avaliacaoRecall(True,10)

In [34]:
avaliacaoRecall(True,20)

Recall: 0.7322033898305085
Duração: 26.240152


### Precision

In [73]:
avaliacaoPrecision(True,5)

Precision: 0.10644067796610142


In [74]:
avaliacaoPrecision(True,10)

Precision: 0.0603389830508474


In [75]:
avaliacaoPrecision(True,20)

Precision: 0.033220338983050865


### MAP

In [76]:
avaliacaoMAP(True,5)

MAP@ 5: 0.002793


In [77]:
avaliacaoMAP(True,10)

MAP@ 10: 0.001935


In [78]:
avaliacaoMAP(True,20)

MAP@ 20: 0.001273


### RR

In [79]:
avaliacaoRR(True)

RR: 0.362712


## Config 21

In [35]:
index_path = 'E:\Corpora_em_json\pyterrier\pre_processamento_unigram_bigram_savoy_full'

In [36]:
retriever = pt.BatchRetrieve(index_path,{"vmodel":"BM25","k1":1.5,"b":0.75})

In [37]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = Savoy()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + "_" + w[1]
        ngram.append(string)
    
    return " ".join(ngram)

## Avaliação

### Recall

In [ ]:
avaliacaoRecall(True,5)

In [ ]:
avaliacaoRecall(True,10)

In [38]:
avaliacaoRecall(True,20)

Recall: 0.7186440677966102
Duração: 23.426928


### Precision

In [86]:
avaliacaoPrecision(True,5)

Precision: 0.10915254237288108


In [87]:
avaliacaoPrecision(True,10)

Precision: 0.0603389830508474


In [88]:
avaliacaoPrecision(True,20)

Precision: 0.032372881355932207


### MAP

In [89]:
avaliacaoMAP(True,5)

MAP@ 5: 0.003359


In [90]:
avaliacaoMAP(True,10)

MAP@ 10: 0.001985


In [91]:
avaliacaoMAP(True,20)

MAP@ 20: 0.001364


### RR

In [92]:
avaliacaoRR(True)

RR: 0.369492


## Config 22

In [39]:
index_path = 'E:\Corpora_em_json\pyterrier\pre_processamento_rslps_full'

In [40]:
retriever = pt.BatchRetrieve(index_path,{"vmodel":"BM25","k1":1.5,"b":0.75})

In [41]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLP_S()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    return " ".join(terms)

## Avaliação

### Recall

In [42]:
avaliacaoRecall(True,5)

Recall: 0.6067796610169491
Duração: 20.985567


In [43]:
avaliacaoRecall(True,10)

Recall: 0.6338983050847458
Duração: 16.770001


In [44]:
avaliacaoRecall(True,20)

Recall: 0.7016949152542373
Duração: 16.782034


### Precision

In [22]:
avaliacaoPrecision(True,5)

Precision: 0.10576271186440651


In [23]:
avaliacaoPrecision(True,10)

Precision: 0.058644067796610085


In [24]:
avaliacaoPrecision(True,20)

Precision: 0.03203389830508474


### MAP

In [25]:
avaliacaoMAP(True,5)

MAP@ 5: 0.002985


In [26]:
avaliacaoMAP(True,10)

MAP@ 10: 0.001898


In [27]:
avaliacaoMAP(True,20)

MAP@ 20: 0.001218


### RR

In [28]:
avaliacaoRR(True)

RR: 0.369492


## Config 23

In [45]:
index_path = 'E:\Corpora_em_json\pyterrier\pre_processamento_unigram_bigram_rslps_full'

In [46]:
retriever = pt.BatchRetrieve(index_path,{"vmodel":"BM25","k1":1.5,"b":0.75})

In [47]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLP_S()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + "_" + w[1]
        ngram.append(string)
    
    return " ".join(ngram)

## Avaliação

### Recall

In [ ]:
avaliacaoRecall(True,5)

In [ ]:
avaliacaoRecall(True,10)

In [48]:
avaliacaoRecall(True,20)

Recall: 0.6983050847457627
Duração: 23.268292


### Precision

In [35]:
avaliacaoPrecision(True,5)

Precision: 0.10644067796610142


In [36]:
avaliacaoPrecision(True,10)

Precision: 0.05898305084745755


In [37]:
avaliacaoPrecision(True,20)

Precision: 0.03152542372881355


### MAP

In [38]:
avaliacaoMAP(True,5)

MAP@ 5: 0.003479


In [39]:
avaliacaoMAP(True,10)

MAP@ 10: 0.001918


In [40]:
avaliacaoMAP(True,20)

MAP@ 20: 0.001250


### RR

In [41]:
avaliacaoRR(True)

RR: 0.362712
